<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-constants" data-toc-modified-id="Define-constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define constants</a></span><ul class="toc-item"><li><span><a href="#Define-input-paths" data-toc-modified-id="Define-input-paths-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Define input paths</a></span></li><li><span><a href="#Init-scalers-dict" data-toc-modified-id="Init-scalers-dict-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Init scalers dict</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Load data</a></span></li></ul></li><li><span><a href="#Define-scalers" data-toc-modified-id="Define-scalers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define scalers</a></span><ul class="toc-item"><li><span><a href="#Define-all-possible-labels-for-the-categorical-scalers" data-toc-modified-id="Define-all-possible-labels-for-the-categorical-scalers-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Define all possible labels for the categorical scalers</a></span></li><li><span><a href="#Default-assignement" data-toc-modified-id="Default-assignement-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Default assignement</a></span></li><li><span><a href="#Init-scalers" data-toc-modified-id="Init-scalers-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Init scalers</a></span></li></ul></li><li><span><a href="#Check-Scalers" data-toc-modified-id="Check-Scalers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Check Scalers</a></span></li><li><span><a href="#Save-scalers" data-toc-modified-id="Save-scalers-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Save scalers</a></span></li><li><span><a href="#Try-Scalers" data-toc-modified-id="Try-Scalers-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Try Scalers</a></span></li></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from arrow_utils import *
from arrow_clouds.data_scalers import *

/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


# Define constants

In [12]:
VOLUME = 50
RES='5S'

## Define input paths

In [13]:
dataset_folder = '../training/data/data_vol{}_bar_{}/'.format(VOLUME, RES)
data_saver = DataSaver(dataset_folder)

## Init scalers dict

In [14]:
scalers = {'CategoricalScaler': [], 'Identity': [], 'StretchedScaler': [], 'MinMaxScaler': [], 'BoxCoxTransformXGB': [], 'BoxCoxTransform': [],
          'StandardScaler': []} # default scalers dict
categorical_classes = {} # categorical scaler labels
scalers_dict = {} # final scalers dict

## Load data

In [15]:
data = data_saver.load_df()
fdump = data_saver.load_fdump()
feats = fdump['feats']

data=data[fdump['all_feats'] + fdump['target_cols']]

In [16]:
fdump['feats'].keys()

dict_keys(['fd_prices', 'range', 'prices', 'sigma', 'cb_vola', 'deviation', 'differential', 'indicators', 'duration', 'calendar', 'nlp'])

# Define scalers

## Define all possible labels for the categorical scalers

In [17]:
categorical_classes['dow'] = np.arange(0, 7, 1)
categorical_classes['day'] = np.arange(1, 32, 1)
categorical_classes['wom'] = np.arange(1, 6, 1)
categorical_classes['month'] = np.arange(1, 13, 1)
categorical_classes['elapsed_minutes'] = np.arange(0, 24 * 60 + 1, 1)
categorical_classes['elapsed_5_minutes'] = np.arange(0, 24 * 60 + 1, 5)

## Default assignement

In [18]:
## Target Columns
scalers['Identity'] += fdump['target_cols']

In [19]:
## Features
for k in ['fd_prices', 'range', 'sigma', 'cb_vola', 'deviation', 'differential',
          'indicators', 'duration']:
    scalers['StretchedScaler'] += fdump['feats'][k]

# prices
fd_smoothed_prices = [f for f in fdump['feats']['prices'] if 'FD_VWAP' in f]
smoothed_prices = list(set(fdump['feats']['prices'] ) - set(fd_smoothed_prices))
scalers['StretchedScaler'] += fd_smoothed_prices
scalers['MinMaxScaler'] += smoothed_prices

In [20]:
# NLP
scalers['Identity'] += fdump['feats']['nlp']

In [21]:
# Calendar Feats
scalers['CategoricalScaler'] += ['dow','day', 'wom', 'month', 'elapsed_minutes', 'elapsed_5_minutes']
scalers['StretchedScaler'] += ['dt_time', 'elapsed_minutes_cont']

## Init scalers

In [22]:
# Initalize default scalers
for col in data.columns:
    if col in scalers['BoxCoxTransformXGB']:
        scalers_dict[col] = BoxCoxTransformXGB(StretchedScaler(stretch=1, input_perc_clipping=(0.5, 99.5)))

    if col in scalers['BoxCoxTransform']:
        scalers_dict[col] = BoxCoxTransform(StretchedScaler(stretch=1, input_perc_clipping=(0.5, 99.5)))

    if col in scalers['StretchedScaler']:
        scalers_dict[col] = StretchedScaler(output_clipping_values=(-2,2))

    if col in scalers['CategoricalScaler']:
        scalers_dict[col] = CategoricalScaler(class_categories=categorical_classes[col])
        
    if col in scalers['StandardScaler']:
        scalers_dict[col] = StandardScaler()

    if col in scalers['MinMaxScaler']:
        scalers_dict[col] = MinMaxScaler()
        
    if col in scalers['Identity']:
        scalers_dict[col] = Identity()

    elif col not in scalers_dict:
        print('No scaler is set for column {}!'.format(col))

# Check Scalers

In [24]:
from cqcplot import *

/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [27]:
# for k in ['range']:
#     if k!='calendar':
#         p = variable_histogram(data.sample(frac=0.1),
#                                feats[k],
#                                transformations=[StretchedScaler()],
#                                fontsize=6,
#                                figsize=(8,len( feats[k])))
#         p['fig'].suptitle(k)

# Save scalers

In [28]:
data_saver.save_scalers(scalers_dict)

# Try Scalers

In [17]:
scalers_to_try = scalers_dict.copy()
df = data.copy()

In [18]:
for col, s in scalers_to_try.items():
    
    fitted_scaler =s.fit(df[col].values.reshape(-1, 1))
    
    df[col] = s.transform(df[col].values.reshape(-1, 1))
    

/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype timedelta64[ns] was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [19]:
# all_feats = scalers['StretchedScaler'] + scalers['MinMaxScaler'] + scalers['StandardScaler']
# fs = getGridShape(len(all_feats))
# p = variable_histogram(df[all_feats].sample(frac=0.1),
#                        all_feats,
#                        fontsize=6,
#                        figsize=(3*fs[1], 2*fs[0]))
# p['fig'].savefig(dataset_folder+'all_feats.png', bbox_inches='tight')

In [30]:
scalers_dict

{'CrossBarVol_VWAP_100R': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a23278>,
 'RelDelta_EWMA5R_VWAP': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a23550>,
 'LogRet_1R_VWAP': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a23588>,
 'MACD200R10R_Close': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a231d0>,
 'EWMA25R_NormCORange': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a234a8>,
 'elapsed_5_minutes': <arrow_clouds.data_scalers.CategoricalScaler at 0x7f5817a23128>,
 'VolatilityDelta_EWMA50R_VWAP': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a230f0>,
 'SI_MPC_50': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a232e8>,
 'Sharpe_50R_Close': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a235c0>,
 'Sharpe_50R_FD_VWAP': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a23630>,
 'sigma_EWMA25R_VWAP': <arrow_clouds.data_scalers.StretchedScaler at 0x7f5817a234e0>,
 'Sharpe_10R_VWAP': <arrow_clouds.data_scalers.Stretc

In [34]:
train = data.iloc[0:100, :]

In [37]:
scalers_dict['CrossBarVol_VWAP_100R'].fit(train['CrossBarVol_VWAP_100R'].reshape(-1, 1))

/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
